In [16]:
# Cell 1 – Imports (no pip commands!)
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
print("All libraries ready!")

All libraries ready!


In [37]:
# Cell 2 – Load your data
df = pd.read_csv('data/processed/reviews_with_sentiment.csv')

# Ignore date processing
# df['review_date'] = pd.to_datetime(df['review_date'], errors='coerce').dt.date

print(f"Loaded {len(df)} reviews from Task 2")
print(df[['bank_name', 'rating', 'sentiment_label']].head())

Loaded 1200 reviews from Task 2
           bank_name  rating sentiment_label
0  Bank of Abyssinia       5         neutral
1  Bank of Abyssinia       5        positive
2  Bank of Abyssinia       5         neutral
3  Bank of Abyssinia       5        positive
4  Bank of Abyssinia       5        positive


In [42]:
# Cell 3 – Connect to your database
conn = psycopg2.connect(
    host="localhost",
    database="bank_reviews",
    user="postgres",        # change to "admin" if you use that user
    password="Hottiefy#1",
    port=5433     # change if your password is different
)
cur = conn.cursor()
print("Connected to bank_reviews database")

Connected to bank_reviews database


In [43]:
banks = [
    ("Commercial Bank of Ethiopia", "CBE Birr"),
    ("Bank of Abyssinia", "Abyssinia Mobile"),
    ("Dashen Bank", "Amole / Dashen Mobile")
]

try:
    # Insert data into the Banks table
    for name, app in banks:
        cur.execute("""
            INSERT INTO Banks (bank_name, app_name)
            VALUES (%s, %s)
            ON CONFLICT (bank_name) DO NOTHING;  -- Avoid conflicts
        """, (name, app))
    
    conn.commit()  # Commit the transaction
    print("3 banks inserted")
except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()  # Roll back the transaction on error
finally:
    cur.close()  # Close the cursor
    

An error occurred: relation "banks" does not exist
LINE 2:             INSERT INTO Banks (bank_name, app_name)
                                ^



In [40]:
# Get bank_id mapping
try:
    cur.execute("SELECT bank_id, bank_name FROM \"Banks\"")
    bank_map = {name: bid for bid, name in cur.fetchall()}
    print("Bank mapping:", bank_map)
except Exception as e:
    print(f"An error occurred while fetching bank ID mapping: {e}")

An error occurred while fetching bank ID mapping: cursor already closed
